In [ ]:
from google.colab import drive
import pandas as pd
from IPython.display import display
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from torch.utils.data import DataLoader, TensorDataset

drive.mount('/content/drive')

old_churches_1 = '/content/drive/MyDrive/OregonChurches/Old_Churches_Oregon_1.csv'
old_churches_2 = '/content/drive/MyDrive/OregonChurches/Old_Churches_Oregon_2.csv'
old_churches_3 = '/content/drive/MyDrive/OregonChurches/Old_Churches_Oregon_3.csv'
latest_churches = '/content/drive/MyDrive/OregonChurches/Churches_Latest.csv'

df_old_1 = pd.read_csv(old_churches_1)
df_old_2 = pd.read_csv(old_churches_2)
df_old_3 = pd.read_csv(old_churches_3)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_old_1.columns = df_old_1.columns.str.strip()
df_old_2.columns = df_old_2.columns.str.strip()
df_old_3.columns = df_old_3.columns.str.strip()

common_columns = ['Business Name', 'Entity Type', 'Nonprofit Type', 'Registry Date',
                  'Associated Name Type', 'Address', 'City', 'State', 'Zip Code']

common_columns_1 = [col for col in common_columns if col in df_old_1.columns]
common_columns_2 = [col for col in common_columns if col in df_old_2.columns]
common_columns_3 = [col for col in common_columns if col in df_old_3.columns]

df_old_1_filtered = df_old_1[common_columns_1]
df_old_2_filtered = df_old_2[common_columns_2]
df_old_3_filtered = df_old_3[common_columns_3]
df_old_merged = pd.concat([df_old_1_filtered, df_old_2_filtered, df_old_3_filtered], ignore_index=True)

df_latest = pd.read_csv(latest_churches)
df_latest.columns = df_latest.columns.str.strip()
df_latest_renamed = df_latest.rename(columns={
    'NAME': 'Business Name',
    'STREET': 'Address',
    'CITY': 'City',
    'STATE': 'State',
    'ZIP': 'Zip Code',
})

df_latest_filtered = df_latest_renamed[[col for col in common_columns if col in df_latest_renamed.columns]]
df_latest_filtered = df_latest_filtered[df_latest_filtered['Business Name'].str.contains('church', case=False, na=False)]
df_latest_filtered_file = '/content/drive/MyDrive/OregonChurches/Churches_Latest.csv'
df_latest_filtered.to_csv(df_latest_filtered_file, index=False)

display(df_old_merged.head())
display(df_latest_filtered.head())

,Business Name,Entity Type,Nonprofit Type,Registry Date,Associated Name Type,Address,City,State,Zip Code
0,THURSTON CHRISTIAN CHURCH,DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1892-12-01 00:00:00,PRINCIPAL PLACE OF BUSINESS,6680 THURSTON RD,SPRINGFIELD,OR,97478
1,CENTENNIAL CHRISTIAN CENTER,DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1897-04-23 00:00:00,PRINCIPAL PLACE OF BUSINESS,1020 JANUS ST,SPRINGFIELD,OR,97477
2,"FIRST BAPTIST CHURCH OF SPRINGFIELD, OREGON",DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1914-11-07 00:00:00,PRINCIPAL PLACE OF BUSINESS,1175 G ST,SPRINGFIELD,OR,97477
3,NEW LIFE CHURCH FIRST ASSEMBLY OF GOD SPRINGFIELD,DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1940-12-30 00:00:00,PRINCIPAL PLACE OF BUSINESS,2080 N 19TH ST,SPRINGFIELD,OR,97477
4,FIRST CHRISTIAN CHURCH OF SPRINGFIELD,DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1942-09-24 00:00:00,PRINCIPAL PLACE OF BUSINESS,395 W CENTENNIAL BLVD,SPRINGFIELD,OR,97477


,Business Name,Address,City,State,Zip Code
0,HILLSBORO VINEYARD CHURCH,511 SW 211TH AVE,BEAVERTON,OR,97006-6420
1,SKYLINE BAPTIST CHURCH,4787 SKYLINE RD S,SALEM,OR,97306-2404
2,VERTICAL CHURCH,PO BOX 707,LA PINE,OR,97739-0707
3,CREEKSIDE COMMUNITY CHURCH,PO BOX 1710,SILVERTON,OR,97381-0365
4,MIRROR POND FREE METHODIST CHURCH,PO BOX 214,BEND,OR,97709-0214


In [ ]:
remaining_churches = df_old_merged[df_old_merged['Business Name'].isin(df_latest_filtered['Business Name'])]
remaining_churches_cleaned = remaining_churches.drop_duplicates(subset=['Business Name'])
remaining_churches_file = '/content/drive/MyDrive/OregonChurches/Remaining_Churches_Cleaned.csv'
remaining_churches_cleaned.to_csv(remaining_churches_file, index=False)
display(remaining_churches_cleaned.head())

churches_no_longer_exist = df_old_merged[~df_old_merged['Business Name'].isin(df_latest_filtered['Business Name'])]
churches_no_longer_exist_cleaned = churches_no_longer_exist.drop_duplicates(subset=['Business Name'])
no_longer_exist_file = '/content/drive/MyDrive/OregonChurches/Churches_No_Longer_Exist_Cleaned.csv'
churches_no_longer_exist_cleaned.to_csv(no_longer_exist_file, index=False)
display(churches_no_longer_exist_cleaned.head())

oregon_businesses_file = '/content/drive/MyDrive/OregonChurches/eo_or.csv'
oregon_business_dataframe = pd.read_csv(oregon_businesses_file)

,Business Name,Entity Type,Nonprofit Type,Registry Date,Associated Name Type,Address,City,State,Zip Code
4,FIRST CHRISTIAN CHURCH OF SPRINGFIELD,DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1942-09-24 00:00:00,PRINCIPAL PLACE OF BUSINESS,395 W CENTENNIAL BLVD,SPRINGFIELD,OR,97477
8,TWIN RIVERS CHURCH,DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1950-09-08 00:00:00,PRINCIPAL PLACE OF BUSINESS,1660 MOHAWK BLVD,SPRINGFIELD,OR,97477
11,SPRINGFIELD CHURCH OF GOD,DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1951-09-28 00:00:00,PRINCIPAL PLACE OF BUSINESS,1369 D ST,SPRINGFIELD,OR,97477
13,CITY OF DESTINY CHURCH,DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1952-11-24 00:00:00,PRINCIPAL PLACE OF BUSINESS,2065 CENTENNIAL BLVD,SPRINGFIELD,OR,97477
16,CAMP CREEK CHURCH,DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1954-03-02 00:00:00,PRINCIPAL PLACE OF BUSINESS,37535 UPPER CAMP CREEK RD,SPRINGFIELD,OR,97478


,Business Name,Entity Type,Nonprofit Type,Registry Date,Associated Name Type,Address,City,State,Zip Code
0,THURSTON CHRISTIAN CHURCH,DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1892-12-01 00:00:00,PRINCIPAL PLACE OF BUSINESS,6680 THURSTON RD,SPRINGFIELD,OR,97478
1,CENTENNIAL CHRISTIAN CENTER,DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1897-04-23 00:00:00,PRINCIPAL PLACE OF BUSINESS,1020 JANUS ST,SPRINGFIELD,OR,97477
2,"FIRST BAPTIST CHURCH OF SPRINGFIELD, OREGON",DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1914-11-07 00:00:00,PRINCIPAL PLACE OF BUSINESS,1175 G ST,SPRINGFIELD,OR,97477
3,NEW LIFE CHURCH FIRST ASSEMBLY OF GOD SPRINGFIELD,DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1940-12-30 00:00:00,PRINCIPAL PLACE OF BUSINESS,2080 N 19TH ST,SPRINGFIELD,OR,97477
5,"TRINITY BAPTIST CHURCH OF SPRINGFIELD, OREGON",DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1945-03-22 00:00:00,PRINCIPAL PLACE OF BUSINESS,1162 B ST,SPRINGFIELD,OR,97477


In [ ]:
remaining_churches = pd.read_csv('/content/drive/MyDrive/OregonChurches/Remaining_Churches_Cleaned.csv')
churches_no_longer_exist = pd.read_csv('/content/drive/MyDrive/OregonChurches/Churches_No_Longer_Exist_Cleaned.csv')

remaining_churches['Longevity'] = 1
churches_no_longer_exist['Longevity'] = 0

df = pd.concat([remaining_churches, churches_no_longer_exist], ignore_index=True)

label_encoder = LabelEncoder()
df['City'] = label_encoder.fit_transform(df['City'])
df['State'] = label_encoder.fit_transform(df['State'])

X = df[['City', 'State', 'Zip Code']]  # Include more features as needed
y = df['Longevity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)  # Make it (N, 1) for binary classification
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

class ChurchLongevityNN(nn.Module):
    def __init__(self):
        super(ChurchLongevityNN, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

# Initialize the model, loss function, and optimizer
model = ChurchLongevityNN()
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# The accuracy plateus at 50.
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        predicted = (outputs > 0.5).float()  # Convert sigmoid output to binary (0 or 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Accuracy: {accuracy * 100:.2f}%')

Epoch [1/50], Loss: 0.5835
Epoch [2/50], Loss: 0.4621
Epoch [3/50], Loss: 0.3796
Epoch [4/50], Loss: 0.3629
Epoch [5/50], Loss: 0.3606
Epoch [6/50], Loss: 0.3609
Epoch [7/50], Loss: 0.3559
Epoch [8/50], Loss: 0.3712
Epoch [9/50], Loss: 0.3608
Epoch [10/50], Loss: 0.3609
Epoch [11/50], Loss: 0.3639
Epoch [12/50], Loss: 0.3603
Epoch [13/50], Loss: 0.3604
Epoch [14/50], Loss: 0.3653
Epoch [15/50], Loss: 0.3596
Epoch [16/50], Loss: 0.3592
Epoch [17/50], Loss: 0.3653
Epoch [18/50], Loss: 0.3561
Epoch [19/50], Loss: 0.3607
Epoch [20/50], Loss: 0.3605
Epoch [21/50], Loss: 0.3698
Epoch [22/50], Loss: 0.3594
Epoch [23/50], Loss: 0.3650
Epoch [24/50], Loss: 0.3649
Epoch [25/50], Loss: 0.3552
Epoch [26/50], Loss: 0.3638
Epoch [27/50], Loss: 0.3562
Epoch [28/50], Loss: 0.3646
Epoch [29/50], Loss: 0.3606
Epoch [30/50], Loss: 0.3557
Epoch [31/50], Loss: 0.3557
Epoch [32/50], Loss: 0.3610
Epoch [33/50], Loss: 0.3683
Epoch [34/50], Loss: 0.3607
Epoch [35/50], Loss: 0.3682
Epoch [36/50], Loss: 0.3693
E